# Mental Health Program Discharge Survival Analysis - Data EDA
Xiaoyu Yang
<br>
Created on 4/7/24

In [1]:
import datetime 
from functools import reduce
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 2000)
pd.options.mode.copy_on_write = True

### Load data

In [2]:
df = pd.read_csv('./data/analytics_case_study_raw_data (3).csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 69 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   APPOINTMENT_ID_PRIMARY_KEY                                9100 non-null   int64  
 1   APPOINTMENT_DESCRIPTION_LOW_LEVEL_X                       9100 non-null   object 
 2   APPOINTMENT_DATE_X                                        9100 non-null   object 
 3   APPOINTMENT_ATTENDANCE_STATUS_NAME_X                      9100 non-null   object 
 4   EOC_KEY_X                                                 9100 non-null   int64  
 5   APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X                      9100 non-null   object 
 6   Assigned PT                                               9100 non-null   object 
 7   Assigned CS                                               9100 non-null   object 
 8   Assigned CD       

In [4]:
df.isnull().sum()

APPOINTMENT_ID_PRIMARY_KEY                                     0
APPOINTMENT_DESCRIPTION_LOW_LEVEL_X                            0
APPOINTMENT_DATE_X                                             0
APPOINTMENT_ATTENDANCE_STATUS_NAME_X                           0
EOC_KEY_X                                                      0
APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X                           0
Assigned PT                                                    0
Assigned CS                                                    0
Assigned CD                                                    0
EOC_SALESFORCE_OPPORTUNITY_ID_CRM                              0
EOC_INTERESTED_IN_FAMILY_THERAPY_CRM                         381
EOC_FAMILY_THERAPY_SCHEDULED_DURING_BPS_CRM                    0
EOC_FAMILY_THERAPY_NOT_SCHEDULED_DURING_BPS_REASON_CRM      7777
EOC_SCHEDULED_INDIVIDUAL_THERAPY_HOURS_PRODUCT                 0
EOC_SCHEDULED_FAMILY_THERAPY_HOURS_PRODUCT                     0
EOC_REFERRAL_SOURCE_TYPE_

In [5]:
df.head()

,APPOINTMENT_ID_PRIMARY_KEY,APPOINTMENT_DESCRIPTION_LOW_LEVEL_X,APPOINTMENT_DATE_X,APPOINTMENT_ATTENDANCE_STATUS_NAME_X,EOC_KEY_X,APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X,Assigned PT,Assigned CS,Assigned CD,EOC_SALESFORCE_OPPORTUNITY_ID_CRM,EOC_INTERESTED_IN_FAMILY_THERAPY_CRM,EOC_FAMILY_THERAPY_SCHEDULED_DURING_BPS_CRM,EOC_FAMILY_THERAPY_NOT_SCHEDULED_DURING_BPS_REASON_CRM,EOC_SCHEDULED_INDIVIDUAL_THERAPY_HOURS_PRODUCT,EOC_SCHEDULED_FAMILY_THERAPY_HOURS_PRODUCT,EOC_REFERRAL_SOURCE_TYPE_CRM,EOC_REFERRAL_SOURCE_SUBTYPE_CRM,EOC_REFERRAL_SOURCE_STATE_CRM,EOC_OUTREACH_REFERRAL_OWNER_CRM,EOC_HAS_CARE_CENTER_ACCESS_PRODUCT,EOC_NUMBER_OF_CARE_CENTER_ACCOUNTS_WITH_ACCESS_PRODUCT,CLIENT_DATE_OF_BIRTH_CRM,CLIENT_RACE_ETHNICITY_NAME_CRM,CLIENT_RELIGION_NAME_CRM,CLIENT_PREFERRED_LANGUAGE_CRM,CLIENT_GENDER_CRM,CLIENT_PREFERRED_PRONOUN_CRM,CLIENT_PREFERRED_NAME_IS_SAFE_CRM,CLIENT_REFERRAL_SOURCE_CONTACT_NAME_CRM,CLIENT_SEX_EHR,EOC_PROGRAM_TYPE_MH_OR_SUD_CRM,EOC_OPPORTUNITY_CONFIRMED_LEAD_SOURCE_TYPE_CRM,EOC_PAYMENT_PLAN_IS_NEEDED_CRM,EOC_ADMISSION_PENDING_REASON_CRM,EOC_REASON_FOR_DISCHARGE_CRM,CLIENT_ACCOUNT_SOURCE_TYPE_CRM,CLIENT_ACCOUNT_TYPE_NAME_CRM,CLIENT_ACCOUNT_IS_TEST_ACCOUNT_CRM,EOC_PROGRAM_AND_CARE_GROUP_CODE_PRODUCT,EOC_CARE_GROUP_CODE_PRODUCT,EOC_CARE_PROGRAM_ADOL_VS_YA_PRODUCT,EOC_INSURANCE_PRIMARY_OR_SECONDARY_CRM,EOC_INSURANCE_PLAN_TYPE_CRM,EOC_INSURANCE_PAYOR_TYPE_CRM,EOC_INSURANCE_PLAN_NAME_CRM,EOC_INSURANCE_CARRIER_NAME_CRM,CLIENT_STATE_CODE_X,CLIENT_ADDRESS_LINE1_X,CLIENT_CITY_X,CLIENT_ZIP_CODE_X,CLIENT_COUNTRY_X,CLIENT_COUNTRY_CODE_X,CLIENT_PHONE_X,CLIENT_EMAIL_X,CLIENT_FULL_NAME_NO_PREFERRED_NAME_X,CLIENT_FULL_NAME_WITH_PREFERRED_NAME_X,Appointment provider,Appointment provider supervisor,Appointment Provider Race,Appointment Provider employment type,Appointment Provider Charlie Health tenure (years),Appointment Provider Years of experience (years),Appointment Provider Gender,Assigned Primary Therapist Race,Assigned Primary Therapist employment type,Assigned Primary Therapist Charlie Health tenure (years),Assigned Primary Therapist Years of experience (years),Assigned Primary Therapist Gender,Session Notes
0,0,Group Session,12/19/2023,Attended,11138868,IOP,PT 34,CS 7,CD 3,208-39-8583,False,FALSE,NaN,1,0,Outpatient Practice,NaN,Arkansas,Andi Vilaboy,True,1,5/18/2001,White,No Religious Affiliation,English,Female,He/Him,False,Sean Gutierrez,F,Mental Health,Professional Referral,False,NaN,Routine,Professional Referral,Client - Patient,False,ADOL-DLGO17,DGO79,ADOL,Primary,PPO,Commercial,Randall Miller,Jennifer Kramer,Oregon,21109 Barrera Hills Apt. 884,Powersmouth,79517.0,United States,ID,507.622.8033,berrydawn@example.com,Karen Perez,Timothy Smith,Group Facilitator 31,Group Supervisor 7,White,1099,1.0,9.0,Male,unknown,employee_full_time,1,4,Male,The session revolved around the client's anxie...
1,2,Group Session,1/15/2024,Incomplete,10422996,IOP,PT 22,CS 5,CD 2,819-55-7947,True,FALSE,NaN,1,1,Outpatient Practice,NaN,Minnesota,Amanda Davison,True,0,11/21/2009,White,No Religious Affiliation,English,Female,He/Him,False,Victor Bass,M,Mental Health,Internet,False,NaN,Routine,Internet,Client - Patient,False,ADOL-CFTO24,DGY55,ADOL,Primary,OA,Commercial,Pamela Horton,Melissa Newton,Virginia,711 Mclaughlin Via,Hillfort,25045.0,United States,GR,242-447-4376,nicholeking@example.org,Nicholas Mccormick,Benjamin Jackson,Group Facilitator 1,Group Supervisor 1,White,1099,0.0,2.0,Female,Other Race,employee_full_time,2,10,Male,NaN
2,4,Group Session,11/15/2023,Attended,11893825,IOP,PT 42,CS 9,CD 3,873-76-1907,True,FALSE,NaN,1,0,NaN,Hospital System,Minnesota,D Becker,True,1,6/5/2000,Other Race,Christianity,English,Non-Binary,She/Her,False,NaN,Female,Mental Health,Professional Referral,False,NaN,Routine,Professional Referral - Revalidate,Client - Patient,False,ADOL-POCY2,DGO43,YA,Primary,OA+,Commercial,Amy Brown,Caroline Dixon,Alabama,669 Williams Fort,Brownmouth,51288.0,United States,PE,+1-974-547-9409x551,gregoryevans@example.net,Dale Bowers,Nancy

### Basic inspection
After some basic inspection of the data, below is a summary of changes so far:
1. Filtered out test accounts
2. It is possible to use email + birthday to identify a unique client. Although in this data set it is possible to use "EOC_KEY_X" to identify unique clients, it's better to create a client ID for future use
3. Set the correct data types for birthday, appointment date and zip code columns

In [6]:
# filter data for non-test records
df = df[df['CLIENT_ACCOUNT_IS_TEST_ACCOUNT_CRM'] == False].reset_index()

In [7]:
# column names of 4 subsets
cli_cols = [col for col in df.columns if col.startswith('CLIENT')]
app_cols = [col for col in df.columns if col.startswith('APPOINTMENT')]
eoc_cols = [col for col in df.columns if col.startswith('EOC')]
prd_cols = [col for col in df.columns if col not in cli_cols + app_cols + eoc_cols]

In [8]:
# It looks like there are 887 unique clients in the data set, and 'EOC_KEY_X' can represent unique clients
print('Number of unique clients: ', df[[col for col in df.columns if col.startswith('CLIENT')]].drop_duplicates().shape[0])
print('Number of unique clients identified by email + birthday: ', df.drop_duplicates(subset = ['CLIENT_EMAIL_X', 'CLIENT_DATE_OF_BIRTH_CRM']).shape[0])
print('Number of unique episodes of care: ', df.EOC_KEY_X.nunique())

Number of unique clients:  887
Number of unique clients identified by email + birthday:  887
Number of unique episodes of care:  887


In [9]:
# set correct data types to columns
df['client_id'] = df['CLIENT_EMAIL_X'].astype('str') + df['CLIENT_DATE_OF_BIRTH_CRM'].astype('str')
df['CLIENT_DATE_OF_BIRTH_CRM'] = pd.to_datetime(df['CLIENT_DATE_OF_BIRTH_CRM'])
df['APPOINTMENT_DATE_X'] = pd.to_datetime(df['APPOINTMENT_DATE_X'])
df['CLIENT_ZIP_CODE_X'] = df['CLIENT_ZIP_CODE_X'].apply(lambda x: "{:.0f}".format(x) if not pd.isnull(x) else x).astype('str').apply(lambda x: x.zfill(5))

In [10]:
# drop duplicates at client_id + EOC_KEY_X + APPOINTMENT_DATE_X level
df = df.drop_duplicates(subset = ['client_id', 'EOC_KEY_X', 'APPOINTMENT_DATE_X'], keep = 'last')

In [11]:
df.shape

(7596, 71)

### Create target variable
This is a survival analysis and modeling exercise to understand the "survival time" until a discharge happened (whether it's successful or with exception). In order to frame the analytical problem into a survival analysis and modeling one, below are the relevant definitions:
1. The timeline of the data set is from 10/1/23 to 3/5/24, which will be the observation period of this analysis + modeling exercise
2. Discharge: as long as there are values in the column 'EOC_REASON_FOR_DISCHARGE_CRM'. There will be a separate column to denote if it's a successful discharge or with exceptions or other situations
3. Censorship: as long as there are values in the discharge column, the censorship status will be True, otherwise it's False. "True" indicates that we have observed discharge within the timeline we are observing; "False" indicates otherwise, which means the data is right-censored
4. Survival time: defined as the days between the first and the last appointment dates

In [12]:
print('Start time of the analysis:', df.APPOINTMENT_DATE_X.min())
print('End time of the analysis:', df.APPOINTMENT_DATE_X.max())

Start time of the analysis: 2023-10-01 00:00:00
End time of the analysis: 2024-03-05 00:00:00


In [13]:
df.groupby('EOC_REASON_FOR_DISCHARGE_CRM').client_id.nunique().sort_values(ascending = False)

EOC_REASON_FOR_DISCHARGE_CRM
Routine                                             546
Routine with exception (non-attendance)             103
Routine with exception (poor program fit)            75
Therapeutic Discharge, HLOC                          48
Administrative                                       35
Routine with exception (scheduling difficulties)     23
Therapeutic Discharge, LLOC                           6
Insurance denial                                      5
Insurance discharge (non-payment)                     3
Routine with exception (Incarceration)                1
Name: client_id, dtype: int64

In [14]:
# create a column to denote discharge status, which can be used as the censorship status in the target variable
df['discharge_flag'] = np.where(df.EOC_REASON_FOR_DISCHARGE_CRM.isnull(), False, True)

# create a column to denote if there is non-routine discharge
df['non_routine_discharge_flag'] = np.where(
    ~df.discharge_flag, 'NA', np.where(
        df.EOC_REASON_FOR_DISCHARGE_CRM != 'Routine', 'Y', 'N'
    )
)

print('Censorship rate of the data set:', round(df[~df.discharge_flag].client_id.nunique() / df.client_id.nunique() * 100, 2), '%')

Censorship rate of the data set: 4.74 %


In [15]:
# calculate the survival time in days for each client
survival_time = df.groupby(['client_id', 'EOC_KEY_X']).agg(
    start_date = ('APPOINTMENT_DATE_X', 'min'),
    end_date = ('APPOINTMENT_DATE_X', 'max')
).reset_index()

survival_time['survival_days'] = (survival_time.end_date - survival_time.start_date) // np.timedelta64(1, 'D')

In [16]:
# use the censorship status on the latest appointment date as the final stage
df['app_date_rk'] = df.groupby(['client_id', 'EOC_KEY_X']).APPOINTMENT_DATE_X.rank(method = 'dense', ascending = False)
latest_discharge = df[df.app_date_rk == 1.0][['client_id', 'EOC_KEY_X', 'discharge_flag', 'non_routine_discharge_flag']]

In [17]:
# combine survival time and censorship status
df_target = pd.merge(survival_time, latest_discharge, on = ['client_id', 'EOC_KEY_X'])
df_target = df_target.drop(columns = ['start_date', 'end_date']).drop_duplicates(subset = ['client_id', 'EOC_KEY_X'], keep = 'last')

In [18]:
df_target.head()

,client_id,EOC_KEY_X,survival_days,discharge_flag,non_routine_discharge_flag
0,aaron78@example.org7/9/2009,10998928,67,True,N
1,aaronromero@example.com5/25/2011,12319804,6,True,Y
2,abigail98@example.com1/13/2003,11023578,108,True,N
3,abigailanderson@example.net11/9/2005,12173322,57,True,N
4,abridges@example.org8/30/2004,9665140,94,True,N


`df_model_ready` data set is created to extract censorship status and survival days later when combined with X features

### Create feature set
Below are the categories of candidate features that can be derived from the data set:
1. Demographics: age, gender, race, religion flag, Spanish flag
2. Appointments: # appointments in the past, average days between appointments, days since the last appointment, % No Show, % Cancellation, % BPS, % IOP, % OPT
3. Product and services: 
4. Providers and therapists: provider/therapist gender, tenure, part time vs. full time

#### Demographics

In [19]:
# calculate age using the current date
df['age'] = (datetime.datetime.today() - df.CLIENT_DATE_OF_BIRTH_CRM) // np.timedelta64(1, 'Y')

In [20]:
# EHR is the most reliable source of gender, but it is also useful to create a flag to indicate if the client has special gender identity
df.groupby('CLIENT_SEX_EHR').client_id.nunique()

CLIENT_SEX_EHR
F                 442
Female             68
Intersex/Other      3
M                 248
Male               25
N                  74
Unknown            27
Name: client_id, dtype: int64

In [21]:
df.groupby('CLIENT_GENDER_CRM').client_id.nunique()

CLIENT_GENDER_CRM
Female                                      511
Gender Fluid                                  3
Gender Neutral                               29
Genderqueer Gender Nonconforming Neither      7
Male                                        286
Non-Binary                                   21
Other                                         1
Transgender                                  17
Name: client_id, dtype: int64

In [22]:
df.groupby(['CLIENT_SEX_EHR', 'CLIENT_GENDER_CRM']).client_id.nunique()

CLIENT_SEX_EHR  CLIENT_GENDER_CRM                       
F               Female                                      310
                Gender Neutral                                5
                Genderqueer Gender Nonconforming Neither      3
                Male                                        119
                Non-Binary                                    1
                Other                                         1
                Transgender                                   1
Female          Female                                       30
                Gender Fluid                                  2
                Gender Neutral                                4
                Male                                         19
                Non-Binary                                    8
                Transgender                                   2
Intersex/Other  Female                                        1
                Male                           

In [23]:
df['gender'] = np.where(
    (df.CLIENT_SEX_EHR.isin(['F', 'Female'])) & ((df.CLIENT_GENDER_CRM) == 'Female'), 'Female', np.where(
        (df.CLIENT_SEX_EHR.isin(['M', 'Male'])) & ((df.CLIENT_GENDER_CRM) == 'Male'), 'Male', 'Nonbinary'
    )
)

In [24]:
# Pick top 2 ethinicies and create a race feature with less levels
df.groupby('CLIENT_RACE_ETHNICITY_NAME_CRM').client_id.nunique()

CLIENT_RACE_ETHNICITY_NAME_CRM
2 or more                                     40
American Indian or Alaska Native               7
Asian                                         14
Black or African American                    125
Hispanic or Latino                             1
Native Hawaiian or Other Pacific Islander      3
Other Race                                    98
White                                        597
unknown                                        2
Name: client_id, dtype: int64

In [25]:
df['race'] = np.where(
    df.CLIENT_RACE_ETHNICITY_NAME_CRM == 'White', 'White', np.where(
        df.CLIENT_RACE_ETHNICITY_NAME_CRM == 'Black or Africa America', 'Black', 'Other'
    )
)

In [26]:
# Create a binary variable to indicate if a client has religious belief
df.groupby('CLIENT_RELIGION_NAME_CRM').client_id.nunique()

CLIENT_RELIGION_NAME_CRM
Agnostic                     11
Atheist                       4
Baptist                       5
Buddhism                      4
Catholic                     35
Christianity                166
Episcopal                    41
Hinduism                      1
Jehova's Witness              4
Jewish                       21
Judaism                       2
Lutheran                      1
Mormon                        5
Muslim                        2
No Religious Affiliation    562
Nondenominational            11
Pagan                         3
Protestant                    3
Unknown Religion              4
Unstated Religion             1
Name: client_id, dtype: int64

In [27]:
df['religion_ind'] = np.where(df.CLIENT_RELIGION_NAME_CRM == 'No Religious Affiliation', 'N', 'Y')

In [28]:
# Create a binary feature to indicate if a client prefers Spanish
df.groupby('CLIENT_PREFERRED_LANGUAGE_CRM').client_id.nunique()

CLIENT_PREFERRED_LANGUAGE_CRM
English    843
Spanish     37
Name: client_id, dtype: int64

In [29]:
df['spanish_ind'] = np.where(df.CLIENT_PREFERRED_LANGUAGE_CRM == 'Spanish', 'Y', 'N')

In [30]:
# combine the processed columns for modeling. Below are the demographic variables for modeling
df_demographics = df[['client_id', 'EOC_KEY_X', 'age', 'gender', 'race', 'religion_ind', 'spanish_ind']].drop_duplicates(subset = ['client_id', 'EOC_KEY_X'], keep = 'last')

In [31]:
df_demographics.head()

,client_id,EOC_KEY_X,age,gender,race,religion_ind,spanish_ind
31,ann40@example.com4/17/2010,9916458,13,Nonbinary,Other,N,N
184,veronicagraves@example.org2/24/2001,9933007,23,Female,White,N,Y
197,robert73@example.org3/10/2009,9931050,15,Female,White,Y,N
350,murphyjo@example.net1/31/2006,13176964,18,Female,White,N,N
439,elaine99@example.org5/28/2003,9679034,20,Nonbinary,Other,N,N


#### Appointments

In [32]:
# calculate average days between appointments and number of appointments 
df = df.sort_values(by = ['client_id', 'EOC_KEY_X', 'APPOINTMENT_DATE_X'])
df['prev_app_date'] = df.groupby(['client_id', 'EOC_KEY_X']).APPOINTMENT_DATE_X.shift(1)
df['days_from_prev_app'] = (df.APPOINTMENT_DATE_X - df.prev_app_date) // np.timedelta64(1, 'D')
app_stats = df.groupby(['client_id', 'EOC_KEY_X']).agg(
    avg_days_between_apps = ('days_from_prev_app', 'mean'),
    num_apps = ('APPOINTMENT_DATE_X', 'nunique')
).reset_index()

app_stats.fillna(0, inplace = True)

In [33]:
# days from the most recent appointment
latest_app = df[df.app_date_rk == 1][['client_id', 'EOC_KEY_X', 'days_from_prev_app']]
app_stats = pd.merge(app_stats, latest_app, 'left', on = ['client_id', 'EOC_KEY_X'])

In [34]:
df.groupby(['APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X', 'APPOINTMENT_DESCRIPTION_LOW_LEVEL_X']).client_id.nunique()

APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X  APPOINTMENT_DESCRIPTION_LOW_LEVEL_X  
BPS                                   BPS                                      293
IOP                                   Group Session                            860
OPT                                   Family Therapy With Client Present        80
                                      Family Therapy Without Client Present     16
                                      Individual Session                       542
OPT - Not billed                      Crisis Call                               18
Other                                 Unknown                                  265
Psych                                 Psychiatric Evaluation                    24
                                      Psychiatric Follow-Up                     31
Name: client_id, dtype: int64

In [35]:
# % bu appointment status 
df['app_status_type'] = np.where(df.APPOINTMENT_ATTENDANCE_STATUS_NAME_X == 'Incomplete', 'Attended', df.APPOINTMENT_ATTENDANCE_STATUS_NAME_X)
app_status_type = df.groupby(['client_id', 'EOC_KEY_X', 'app_status_type']).agg(num_app_type = ('APPOINTMENT_DATE_X', 'nunique')).reset_index()
app_status_type = pd.merge(app_status_type, app_stats[['client_id', 'EOC_KEY_X', 'num_apps']], 'left', on = ['client_id', 'EOC_KEY_X'])
app_status_type['perc_visits'] = app_status_type.num_app_type / app_status_type.num_apps
app_status_type = app_status_type.pivot(index = ['client_id', 'EOC_KEY_X'], columns = 'app_status_type', values = 'perc_visits').reset_index().rename(columns = {'Cancelled': 'cancelled', 'No Show': 'no_show'}).drop(columns = ['Attended'])
app_status_type.fillna(0, inplace = True)

In [36]:
# % by visit type
df['app_type'] = np.where(df.APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X.isin(['BPS', 'IOP', 'OPT']), df.APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X, 'other_visit')
app_visit_type = df.groupby(['client_id', 'EOC_KEY_X', 'app_type']).agg(num_visit_type = ('APPOINTMENT_DATE_X', 'nunique')).reset_index()
app_visit_type = pd.merge(app_visit_type, app_stats[['client_id', 'EOC_KEY_X', 'num_apps']], 'left', on = ['client_id', 'EOC_KEY_X'])
app_visit_type['perc_visits'] = app_visit_type.num_visit_type / app_visit_type.num_apps
app_visit_type = app_visit_type.pivot(index = ['client_id', 'EOC_KEY_X'], columns = 'app_type', values = 'perc_visits').reset_index().drop(columns = 'other_visit')
app_visit_type.fillna(0, inplace = True)

In [37]:
# combine the stats
app_types = pd.merge(app_status_type[['client_id', 'EOC_KEY_X', 'cancelled', 'no_show']], app_visit_type[['client_id', 'EOC_KEY_X', 'BPS', 'IOP', 'OPT']], 'inner', on = ['client_id', 'EOC_KEY_X'])
app_stats = pd.merge(app_stats, app_types, 'left', on = ['client_id', 'EOC_KEY_X']).drop_duplicates(subset = ['client_id', 'EOC_KEY_X'], keep = 'last')

In [38]:
app_stats.head()

,client_id,EOC_KEY_X,avg_days_between_apps,num_apps,days_from_prev_app,cancelled,no_show,BPS,IOP,OPT
0,aaron78@example.org7/9/2009,10998928,7.444444,10,4.0,0.000000,0.100000,0.100000,0.900000,0.000000
1,aaronromero@example.com5/25/2011,12319804,6.000000,2,6.0,0.000000,0.000000,0.000000,0.000000,1.000000
2,abigail98@example.com1/13/2003,11023578,6.000000,19,14.0,0.052632,0.263158,0.052632,0.736842,0.157895
3,abigailanderson@example.net11/9/2005,12173322,7.125000,9,6.0,0.000000,0.222222,0.000000,0.888889,0.111111
4,abridges@example.org8/30/2004,9665140,23.500000,5,26.0,0.000000,0.400000,0.400000,0.600000,0.000000


#### Product, services and client behaviors

In [39]:
# transform a few more columns
df['opt_hours_scheduled'] = df['EOC_SCHEDULED_INDIVIDUAL_THERAPY_HOURS_PRODUCT'] + df['EOC_SCHEDULED_FAMILY_THERAPY_HOURS_PRODUCT']
df['opt_interest_ind'] = np.where(df['EOC_INTERESTED_IN_FAMILY_THERAPY_CRM'] == True, 'Y', 'N')
df['access_portal_ind'] = np.where(df['EOC_HAS_CARE_CENTER_ACCESS_PRODUCT'] == True, 'Y', 'N')
df['referral_source'] = np.where(df['EOC_REFERRAL_SOURCE_TYPE_CRM'].isin(['Hospital', 'Outpatient Practice']), df['EOC_REFERRAL_SOURCE_TYPE_CRM'], 'Other Referral')
df['client_ins_type'] = np.where(df['EOC_INSURANCE_PLAN_TYPE_CRM'].isin(['Medicaid', 'PPO']), df['EOC_INSURANCE_PLAN_TYPE_CRM'], 'Other')
df = df.rename(columns = {'EOC_CARE_PROGRAM_ADOL_VS_YA_PRODUCT': 'ch_product_type'})

In [40]:
df_product = df[['client_id', 'EOC_KEY_X', 'opt_hours_scheduled', 'opt_interest_ind', 'access_portal_ind', 'referral_source', 'client_ins_type']].drop_duplicates(subset = ['client_id', 'EOC_KEY_X'], keep = 'last')

In [41]:
df_product.head()

,client_id,EOC_KEY_X,opt_hours_scheduled,opt_interest_ind,access_portal_ind,referral_source,client_ins_type
254,aaron78@example.org7/9/2009,10998928,1,Y,N,Other Referral,Other
3932,aaronromero@example.com5/25/2011,12319804,1,N,Y,Outpatient Practice,Medicaid
2313,abigail98@example.com1/13/2003,11023578,1,Y,Y,Outpatient Practice,Other
1084,abigailanderson@example.net11/9/2005,12173322,1,Y,Y,Outpatient Practice,Other
556,abridges@example.org8/30/2004,9665140,1,Y,Y,Other Referral,Medicaid


#### Primary and appointment providers

In [42]:
df.groupby('Appointment Provider Gender').client_id.nunique().sort_values(ascending = False)

Appointment Provider Gender
Female       829
Male         738
Nonbinary    408
Name: client_id, dtype: int64

In [43]:
df.groupby('Appointment Provider Race').client_id.nunique().sort_values(ascending = False)

Appointment Provider Race
White                                        841
Asian                                        585
unknown                                      320
Hispanic or Latino                           293
Black or African American                    189
Other Race                                   177
Native Hawaiian or Other Pacific Islander     40
Name: client_id, dtype: int64

In [44]:
# % appointment provider by employment type
df.groupby(['Appointment Provider employment type']).client_id.nunique()

Appointment Provider employment type
1099                  875
employee_full_time    400
employee_part_time    101
Name: client_id, dtype: int64

In [45]:
# calculate % appointment providers with the same gender identity as the client
df['app_prd_same_gender'] = np.where(df['Appointment Provider Gender'] == df.gender, 1, 0)

# calculate % appointment providers by race
df['app_prd_white'] = np.where(df['Appointment Provider Race'] == 'White', 1, 0)
df['app_prd_asian'] = np.where(df['Appointment Provider Race'] == 'Asian', 1, 0)
df['app_prd_black'] = np.where(df['Appointment Provider Race'] == 'Black or African American', 1, 0)
df['app_prd_hispanic'] = np.where(df['Appointment Provider Race'] == 'Hispanic or Latino', 1, 0)

df['app_prd_full_time'] = np.where(df['Appointment Provider employment type'] == 'employee_full_time', 1, 0)
df['app_prd_part_time'] = np.where(df['Appointment Provider employment type'] == 'employee_part_time', 1, 0)
df['app_prd_contractor'] = np.where(df['Appointment Provider employment type'] == '1099', 1, 0)

# aggregate appointment provider info
app_prd_stats = df.groupby(['client_id', 'EOC_KEY_X']).agg(
    app_prd_same_gender = ('app_prd_same_gender', 'mean'),
    app_prd_white = ('app_prd_white', 'mean'),
    app_prd_black = ('app_prd_black', 'mean'),
    app_prd_asian = ('app_prd_asian', 'mean'),
    app_prd_hispanic = ('app_prd_hispanic', 'mean'),
    app_prd_full_time = ('app_prd_full_time', 'mean'),
    app_prd_part_time = ('app_prd_part_time', 'mean'),
    app_prd_contractor = ('app_prd_contractor', 'mean'),
    app_prd_median_tenure = ('Appointment Provider Years of experience (years)', 'median'),
    app_prd_median_tenure_ch = ('Appointment Provider Charlie Health tenure (years)', 'median'),
).reset_index()

In [46]:
df.groupby('Assigned Primary Therapist Gender').client_id.nunique().sort_values(ascending = False)

Assigned Primary Therapist Gender
Female       615
Male         254
Nonbinary     18
Name: client_id, dtype: int64

In [47]:
df.groupby('Assigned Primary Therapist Race').client_id.nunique().sort_values(ascending = False)

Assigned Primary Therapist Race
White                                        537
Hispanic or Latino                           108
Asian                                         74
Black or African American                     71
Other Race                                    43
Native Hawaiian or Other Pacific Islander     37
unknown                                       17
Name: client_id, dtype: int64

In [48]:
df.groupby('Assigned Primary Therapist employment type').client_id.nunique().sort_values(ascending = False)

Assigned Primary Therapist employment type
1099                  507
employee_full_time    235
employee_part_time    145
Name: client_id, dtype: int64

In [49]:
print('# clients with more than 1 primary provider:', sum(df.groupby('client_id')['Assigned PT'].nunique() > 1))

# clients with more than 1 primary provider: 0


In [50]:
# calculate median tenure of primary providers
pri_prd_stats = df.groupby(['client_id', 'EOC_KEY_X']).agg(
    pri_prd_median_tenure = ('Assigned Primary Therapist Years of experience (years)', 'median'),
    pri_prd_median_tenure_ch = ('Assigned Primary Therapist Charlie Health tenure (years)', 'median'),
).reset_index()

pri_prd = df[['client_id', 'EOC_KEY_X', 'Assigned Primary Therapist Gender', 'Assigned Primary Therapist Race', 'Assigned Primary Therapist employment type']].drop_duplicates(subset = ['client_id', 'EOC_KEY_X'], keep = 'last')\
.rename(columns = {
    'Assigned Primary Therapist Gender': 'pri_prd_gender',
    'Assigned Primary Therapist Race': 'pri_prd_race',
    'Assigned Primary Therapist employment type': 'pri_prd_emp_type'
})

In [51]:
df_prd = pd.merge(app_prd_stats, pri_prd, 'left', on = ['client_id', 'EOC_KEY_X'])
df_prd = pd.merge(df_prd, pri_prd_stats, 'left', on = ['client_id', 'EOC_KEY_X']).drop_duplicates(subset = ['client_id', 'EOC_KEY_X'], keep = 'last')

In [52]:
df_prd.head()

,client_id,EOC_KEY_X,app_prd_same_gender,app_prd_white,app_prd_black,app_prd_asian,app_prd_hispanic,app_prd_full_time,app_prd_part_time,app_prd_contractor,app_prd_median_tenure,app_prd_median_tenure_ch,pri_prd_gender,pri_prd_race,pri_prd_emp_type,pri_prd_median_tenure,pri_prd_median_tenure_ch
0,aaron78@example.org7/9/2009,10998928,0.100000,0.700000,0.0,0.100000,0.100000,0.100000,0.000000,0.900000,5.0,0.5,Male,White,employee_full_time,6.0,1.0
1,aaronromero@example.com5/25/2011,12319804,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,3.0,1.0,Male,Asian,1099,3.0,1.0
2,abigail98@example.com1/13/2003,11023578,0.157895,0.736842,0.0,0.105263,0.000000,0.210526,0.000000,0.736842,5.5,0.5,Female,White,employee_full_time,10.0,2.0
3,abigailanderson@example.net11/9/2005,12173322,0.666667,0.555556,0.0,0.222222,0.111111,0.000000,0.111111,0.888889,6.0,1.0,Female,Native Hawaiian or Other Pacific Islander,employee_part_time,6.0,0.0
4,abridges@example.org8/30/2004,9665140,0.600000,1.000000,0.0,0.000000,0.000000,0.400000,0.000000,0.600000,6.0,1.0,Female,White,employee_part_time,5.0,0.0


### Combine featrues and target variable to create model-ready dataset

In [53]:
# combine all the features and the target df
dfs = [df_target, df_demographics, app_stats, df_product, df_prd]
df_all = reduce(lambda x, y: pd.merge(x, y, 'left', on = ['client_id', 'EOC_KEY_X']), dfs)

# fill in missing values
df_all['days_from_prev_app'] = df_all['days_from_prev_app'].fillna(0.0)
df_all['app_prd_median_tenure'] = df_all['app_prd_median_tenure'].fillna(df_all.app_prd_median_tenure.median())
df_all['app_prd_median_tenure_ch'] = df_all['app_prd_median_tenure_ch'].fillna(df_all.app_prd_median_tenure_ch.median())

In [54]:
print('Shape of the cleaned data set:', df_all.shape)
print('Total # of rows with missing values:', df_all.isnull().sum().sum())

Shape of the cleaned data set: (887, 38)
Total # of rows with missing values: 0


In [55]:
#df_all.to_csv('./data/mh_discharge_sa.csv')